In [1]:
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
import keras

In [3]:
training_data_gen = ImageDataGenerator(
    rescale=1. / 255,
    zoom_range = 0.12,
    height_shift_range = 0.12,
    width_shift_range = 0.12,
    horizontal_flip = True
)
test_data_gen = ImageDataGenerator(rescale=1./255)

In [4]:
train_generator = training_data_gen.flow_from_directory('./data/images/Images',
                                                        target_size=(300, 300),
                                                        batch_size=32,
                                                        class_mode='categorical')
test_generator = test_data_gen.flow_from_directory('./data/test',
                                                        target_size=(300, 300),
                                                        batch_size=32,
                                                        class_mode='categorical')

Found 16666 images belonging to 120 classes.
Found 3911 images belonging to 120 classes.


In [5]:
base_model = keras.applications.MobileNetV2(
    weights='imagenet',
    input_shape=(300,300, 3),
    include_top=False)

for layer in base_model.layers:
    layer.trainable = False

In [12]:
inputs = keras.Input(shape=(300, 300, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(120, activation='softmax')(x)

# add your top layer block to your base model
model = Model(base_model.input, predictions)
print(model.summary())

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 150, 150, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 150, 150, 32  128         ['Conv1[0][0]']                  
                                )                                                           

In [13]:
model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',  # categorical_crossentropy if multi-class classifier
                  metrics=['accuracy'])

In [14]:
model.fit(train_generator,
                        epochs=10,
                        validation_data=test_generator)

Epoch 1/10
521/521 [==============================] - 663s 1s/step - loss: 4.6909 - accuracy: 0.0406 - val_loss: 4.5387 - val_accuracy: 0.0841
Epoch 2/10
521/521 [==============================] - 671s 1s/step - loss: 4.1250 - accuracy: 0.1692 - val_loss: 3.5966 - val_accuracy: 0.2751
Epoch 3/10
521/521 [==============================] - 672s 1s/step - loss: 2.8110 - accuracy: 0.3942 - val_loss: 2.1883 - val_accuracy: 0.5139
Epoch 4/10
521/521 [==============================] - 672s 1s/step - loss: 1.7723 - accuracy: 0.5623 - val_loss: 1.5122 - val_accuracy: 0.6021
Epoch 5/10
521/521 [==============================] - 666s 1s/step - loss: 1.3191 - accuracy: 0.6405 - val_loss: 1.2479 - val_accuracy: 0.6377
Epoch 6/10
521/521 [==============================] - 671s 1s/step - loss: 1.1255 - accuracy: 0.6777 - val_loss: 1.0929 - val_accuracy: 0.6806
Epoch 7/10
521/521 [==============================] - 672s 1s/step - loss: 1.0068 - accuracy: 0.7049 - val_loss: 1.0235 - val_accuracy: 0.6927

In [15]:
model.save('NN/sanford_dog_mobilenetv2.h5')